In [1]:
# 네이버에서 검색어 입력받아 검색 한 후 블로그 메뉴를 선택하고
# 오른쪽에 있는 검색옵션 버튼을 눌러서
# 정렬 방식과 기간을 입력하기

#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys # 시스템
import os  # 시스템

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup    # html 데이터를 전처리
from selenium import webdriver   # 웹 브라우저 자동화
import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
from tqdm import tqdm_notebook   # for문 돌릴 때 진행상황을 %게이지로 알려준다.

In [2]:
# python 버전 확인
!python --version

Python 3.9.10


In [3]:
# 판다스 버전 확인
pd.__version__

'1.4.0'

In [4]:
# 검색어, 검색 기간 정의
query_txt = input('1.크롤링할 키워드는 무엇입니까?: ')
# query_txt2 = input('2.제외할 첫번째 키워드를 입력하세요: ')
# query_txt3 = input('3.제외할 두번째 키워드를 입력하세요: ')

start_date = input('4.조회를 시작할 날짜를 입력하세요(예:20170101) :')
end_date = input('5.조회를 종료할 날짜를 입력하세요(예:20171231): ')

1.크롤링할 키워드는 무엇입니까?: 대전맛집
4.조회를 시작할 날짜를 입력하세요(예:20170101) :20210101
5.조회를 종료할 날짜를 입력하세요(예:20171231): 20210928


In [5]:
query_txt="대전맛집"
start_date="20210101"
end_date="20210928"

# 크롤링할 블로그 url 수집하기

In [6]:
pwd

'/Users/jisookwon/Documents/Crawling/Naver_Blog'

In [15]:
#Step 1. 크롬 웹브라우저 실행
path = "/opt/homebrew/bin/chromedriver"       # 윈도우는 "chromedriver.exe"

driver = webdriver.Chrome(path)
# 사이트 주소는 네이버
driver.get('http://www.naver.com')
time.sleep(2)  # 2초간 정지

In [16]:
pwd

'/Users/jisookwon/Documents/Crawling/Naver_Blog'

In [35]:
#Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_id("query")
element.send_keys(query_txt)  # query_txt는 위에서 입력한 키워드
element.submit()
time.sleep(1)

In [36]:
# 'VIEW' 클릭
driver.find_element_by_link_text("VIEW").click( )
time.sleep(1)

In [37]:
# '블로그' 클릭
driver.find_element_by_link_text("블로그").click( )
time.sleep(1)

In [38]:
# '옵션' 클릭
driver.find_element_by_link_text("옵션").click( )
time.sleep(1)

In [39]:
# 검색옵션 확인
item_li = driver.find_elements_by_css_selector('.option .txt')

for i in range(0, len(item_li)):
    print(item_li[i].text)


전체
블로그
카페
관련도순
최신순
전체
1시간
1일
1주
1개월
3개월
6개월
1년
직접입력옵션펼치기


In [40]:
# 검색기간 '3개월' 클릭
print(item_li[10].text)

item_li[10].click()

3개월


In [41]:
# 스크롤 다운
# driver.execute_script("window.scrollTo(0, 500)")
# time.sleep(2)

# 스크롤을 밑으로 내려주는 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 19431049)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 5
i = 0
while i < 5: # 이 조건이 만족되는 동안 반복 실행
    scroll_down(driver) # 스크롤 다운
    i = i+1

In [42]:
# 블로그 글 url들 수집
url_list = []
title_list = []

# URL_raw 크롤링 시작
articles = ".api_txt_lines.total_tit"
article_raw = driver.find_elements_by_css_selector(articles)
article_raw[0]

<selenium.webdriver.remote.webelement.WebElement (session="47d49a2306a5f673476d53c25cbdea61", element="d1561c6c-af48-40c3-80ad-ace33d3e6672")>

In [43]:
article_raw[0].get_attribute('href')

'https://blog.naver.com/symin67?Redirect=Log&logNo=222695679368'

In [56]:
# 크롤링한 url 정제 시작
for article in article_raw:
    url = article.get_attribute('href')   
    url_list.append(url)

# 1초 쉬어 주고,
time.sleep(1)
    
# 제목 크롤링 시작    
for article in article_raw:
    title = article.text
    title_list.append(title)

    print(title)

print("")
print('url갯수: ', len(url_list))
print('title갯수: ', len(title_list))

홀딱 반한 대전 맛집
힐링했던 대전 맛집 리스트
기억할만한 대전 맛집
대전 맛집, 두부두루치기/칼국수가 맛있는 <광천식당>
심쿵했던 대전 맛집
대전 맛집 육하원칙 반석점, 놀이방이 있는 식당
대전 맛집 금수저은수저,소바공방,오모리생바지락손칼국수,조림
대전맛집 추천 소갈비가 맛있는 청춘식당 가오동맛집으로도 인정
대전 / 괴정동 맛집 / 대전맛집(구 대구막창) / 한민시장 맛집
(대전 맛집/수통골 맛집) 오경자메밀사랑 칼국수
대전 맛집 시민칼국수 정말 맛있게 먹었어요 ^^
진정한 대전 맛집 힘내라 골목상권
나온 대전 맛집의 유성온천 근처 분점 / 대전밥집 / 26개월...
(대전맛집)더리스, 테라베오 & 비래키키(BIRAEKIKI) & song...
봄 제철음식 추천 대전맛집 7
기분좋던 대전 맛집
모두가 반했던 대전 맛집
대전맛집 오월드맛집 청원농장식당 야외에서 먹는 숯불바베큐
한민시장 막창맛집 괴정동 대전맛집
화려했던 대전 맛집
대전맛집 괴정동 롯데백화점 근처 등촌샤브칼국수
대전맛집2
[대전 한민시장 맛집] 괴정동 대전맛집에서 막창모듬 1번~
[대전맛집]소석갈비 맛집'띠울석갈비 본점'
대전 맛집 냄비요리연구소 신메뉴 낙곱새 먹고 왔어요
대전맛집 오류동고기집 대전회... 오류동맛집 양연화로 에서...
줄 서서 먹어야 하는 대전 맛집, 태평소국밥
[대전/맛집] 사이드쇼 신세계대전엑스포점 (대전신세계백화점...
대전 맛집 세컨디포레스트 대전은행점
[대전맛집]회가 신선하고 푸짐한 보물섬수산
[대전 맛집/중리동 맛집] 양평 칼국수 콩국수 *
[대전 유성구] 유성호텔... 즐긴 대전 맛집 :: 26개월 아기랑...
대전맛집ㅣ가족식사 모임 장소 경복궁 대전점
대전 맛집 서가앤쿡 목살한상 2인세트 메뉴, 파스타,필라프 등..
대전 맛집 우리칼국수 두부두루치기 정말 찐!! 강추!!...
[대전맛집] 피자 찐맛집 - 리골레토 시카고피자
[대전맛집]백반기행맛집:대전갈비집
[대전맛집]서대전점/문화동 샐러드 맛집 포케153
대전맛집 대전태평동맛집 대전육사시미 대전국밥집 맛있는곳...

In [57]:
pwd

'/Users/jisookwon/Documents/Crawling/Naver_Blog'

In [58]:
url_list

['https://blog.naver.com/symin67?Redirect=Log&logNo=222695679368',
 'https://blog.naver.com/mmorrismom?Redirect=Log&logNo=222689115119',
 'https://blog.naver.com/lkd0486?Redirect=Log&logNo=222712213135',
 'https://blog.naver.com/angerbong?Redirect=Log&logNo=222703004620',
 'https://blog.naver.com/artlife?Redirect=Log&logNo=222691859558',
 'https://blog.naver.com/sscd0808?Redirect=Log&logNo=222711513123',
 'https://blog.naver.com/skeunhea1?Redirect=Log&logNo=222707525378',
 'https://blog.naver.com/byuli55?Redirect=Log&logNo=222647042431',
 'https://blog.naver.com/anythingg_?Redirect=Log&logNo=222698557558',
 'https://blog.naver.com/dlrltmfr365?Redirect=Log&logNo=222662943718',
 'https://blog.naver.com/pepablue?Redirect=Log&logNo=222710002007',
 'https://blog.naver.com/jeju8253?Redirect=Log&logNo=222676505376',
 'https://blog.naver.com/couchmon?Redirect=Log&logNo=222713981002',
 'https://blog.naver.com/wjdtlsckfud?Redirect=Log&logNo=222687997433',
 'https://blog.naver.com/newosilent?Redi

In [59]:
title_list

['홀딱 반한 대전 맛집',
 '힐링했던 대전 맛집 리스트',
 '기억할만한 대전 맛집',
 '대전 맛집, 두부두루치기/칼국수가 맛있는 <광천식당>',
 '심쿵했던 대전 맛집',
 '대전 맛집 육하원칙 반석점, 놀이방이 있는 식당',
 '대전 맛집 금수저은수저,소바공방,오모리생바지락손칼국수,조림',
 '대전맛집 추천 소갈비가 맛있는 청춘식당 가오동맛집으로도 인정',
 '대전 / 괴정동 맛집 / 대전맛집(구 대구막창) / 한민시장 맛집',
 '(대전 맛집/수통골 맛집) 오경자메밀사랑 칼국수',
 '대전 맛집 시민칼국수 정말 맛있게 먹었어요 ^^',
 '진정한 대전 맛집 힘내라 골목상권',
 '나온 대전 맛집의 유성온천 근처 분점 / 대전밥집 / 26개월...',
 '(대전맛집)더리스, 테라베오 & 비래키키(BIRAEKIKI) & song...',
 '봄 제철음식 추천 대전맛집 7',
 '기분좋던 대전 맛집',
 '모두가 반했던 대전 맛집',
 '대전맛집 오월드맛집 청원농장식당 야외에서 먹는 숯불바베큐',
 '한민시장 막창맛집 괴정동 대전맛집',
 '화려했던 대전 맛집',
 '대전맛집 괴정동 롯데백화점 근처 등촌샤브칼국수',
 '대전맛집2',
 '[대전 한민시장 맛집] 괴정동 대전맛집에서 막창모듬 1번~',
 "[대전맛집]소석갈비 맛집'띠울석갈비 본점'",
 '대전 맛집 냄비요리연구소 신메뉴 낙곱새 먹고 왔어요',
 '대전맛집 오류동고기집 대전회... 오류동맛집 양연화로 에서...',
 '줄 서서 먹어야 하는 대전 맛집, 태평소국밥',
 '[대전/맛집] 사이드쇼 신세계대전엑스포점 (대전신세계백화점...',
 '대전 맛집 세컨디포레스트 대전은행점',
 '[대전맛집]회가 신선하고 푸짐한 보물섬수산',
 '[대전 맛집/중리동 맛집] 양평 칼국수 콩국수 *',
 '[대전 유성구] 유성호텔... 즐긴 대전 맛집 :: 26개월 아기랑...',
 '대전맛집ㅣ가족식사 모임 장소 경복궁 대전점',
 '대전 맛집 서가앤쿡 목살한상 2인세트 메뉴, 파스타,필라프 등..',
 '대전 

In [60]:
df = pd.DataFrame({'url':url_list, 'title':title_list})
df

,url,title
0,https://blog.naver.com/symin67?Redirect=Log&lo...,홀딱 반한 대전 맛집
1,https://blog.naver.com/mmorrismom?Redirect=Log...,힐링했던 대전 맛집 리스트
2,https://blog.naver.com/lkd0486?Redirect=Log&lo...,기억할만한 대전 맛집
3,https://blog.naver.com/angerbong?Redirect=Log&...,"대전 맛집, 두부두루치기/칼국수가 맛있는 <광천식당>"
4,https://blog.naver.com/artlife?Redirect=Log&lo...,심쿵했던 대전 맛집
...,...,...
175,https://blog.naver.com/caitlyn15?Redirect=Log&...,[대전/맛집] 한마음정육식당 유성봉명점 (봉명동 삼겹살...
176,https://blog.naver.com/skana0512?Redirect=Log&...,"[대전맛집] 르몽탁 : 내돈내산, 대전양식, 뇨끼 맛집"
177,https://blog.naver.com/tddrq?Redirect=Log&logN...,[대전맛집] 관평동_청담초밥(가성비맛집)
178,https://blog.naver.com/ksj649?Redirect=Log&log...,대전 맛집 중리동 통일면옥 회냉면 녹두전 조합 최고


In [61]:
pwd

'/Users/jisookwon/Documents/Crawling/Naver_Blog'

In [62]:
# 저장하기
df.to_excel("blog_url.xlsx")

# 블로그 내용 크롤링하기

In [63]:
import sys
import os
import pandas as pd
import numpy as np

In [64]:
# "url_list.csv" 불러오기
url_load = pd.read_excel("blog_url.xlsx") # 기본 모델

num_list = len(url_load)

print(num_list)
url_load

180


,Unnamed: 0,url,title
0,0,https://blog.naver.com/symin67?Redirect=Log&lo...,홀딱 반한 대전 맛집
1,1,https://blog.naver.com/mmorrismom?Redirect=Log...,힐링했던 대전 맛집 리스트
2,2,https://blog.naver.com/lkd0486?Redirect=Log&lo...,기억할만한 대전 맛집
3,3,https://blog.naver.com/angerbong?Redirect=Log&...,"대전 맛집, 두부두루치기/칼국수가 맛있는 <광천식당>"
4,4,https://blog.naver.com/artlife?Redirect=Log&lo...,심쿵했던 대전 맛집
...,...,...,...
175,175,https://blog.naver.com/caitlyn15?Redirect=Log&...,[대전/맛집] 한마음정육식당 유성봉명점 (봉명동 삼겹살...
176,176,https://blog.naver.com/skana0512?Redirect=Log&...,"[대전맛집] 르몽탁 : 내돈내산, 대전양식, 뇨끼 맛집"
177,177,https://blog.naver.com/tddrq?Redirect=Log&logN...,[대전맛집] 관평동_청담초밥(가성비맛집)
178,178,https://blog.naver.com/ksj649?Redirect=Log&log...,대전 맛집 중리동 통일면옥 회냉면 녹두전 조합 최고


In [67]:
dict = {}    # 전체 크롤링 데이터를 담을 그릇

number = 30  # 수집할 글 갯수 정하기

# 수집한 url 돌면서 데이터 수집
for i in tqdm_notebook(range(0, number)):
    # 글 띄우기
    url = url_load['url'][i]
    driver = webdriver.Chrome("chromedriver")  # 윈도우는 "chromedriver.exe"
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    
    try : 
        # iframe 접근
        driver.switch_to.frame('mainFrame')

        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성

        # 제목 크롤링 시작
        overlays = ".se-module.se-module-text.se-title-text"                                 
        tit = driver.find_element_by_css_selector(overlays)          # title
        title = tit.text

        # 글쓴이 크롤링 시작
        overlays = ".nick"                                 
        nick = driver.find_element_by_css_selector(overlays)         # nickname
        nickname = nick.text

        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"                                 
        date = driver.find_element_by_css_selector(overlays)         # datetime
        datetime = date.text

        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)    # contents

        content_list = []
        for content in contents:
            content_list.append(content.text)
 
        content_str = ' '.join(content_list)                         # content_str

        # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
        target_info['content'] = content_str

        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = target_info
        time.sleep(1)
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(i, title)

        # 글 하나 크롤링 후 크롬 창을 닫습니다.
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동합니다.
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 중간,중간에 파일로 저장하기
    if i == 30 or 50 or 80:
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("blog_content.xlsx", encoding='utf-8-sig')
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)

/var/folders/dd/p03cpd850nx3bbxblv922vsm0000gn/T/ipykernel_60995/939174967.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(0, number)):


  0%|          | 0/30 [00:00<?, ?it/s]

1 힐링했던 대전 맛집 리스트
2 기억할만한 대전 맛집
3 대전 맛집, 두부두루치기/칼국수가 맛있는 <광천식당>
4 심쿵했던 대전 맛집
5 대전 맛집 육하원칙 반석점, 놀이방이 있는 식당
6 대전 맛집 금수저은수저,소바공방,오모리생바지락손칼국수,조림


KeyboardInterrupt: 

In [ ]:
dict

In [33]:
# 판다스로 만들기
import pandas as pd
result_df = pd.DataFrame.from_dict(dict, 'index')
result_df

""


In [34]:
# 엑셀로 저장하기
result_df.to_excel("blog_content.xlsx", encoding='utf-8-sig')

In [35]:
pwd

'/Users/jisookwon/Documents/크롤링/네이버블로그 '